<a href="https://colab.research.google.com/github/AKOKEMKada/gee/blob/main/code_telechrger_de_thailand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install earthengine-api
#install geopandas pour lire les shapfile
#!pip install 'geopandas'

In [ ]:
#!earthengine authenticate

In [ ]:
import ee
import pandas as pd
import geopandas as gpd
import folium
import json
from google.colab import drive
import os

In [ ]:
# Authentification auprès de Google Earth Engine
ee.Authenticate()
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com', project="poverty-484600")

In [ ]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


Preparation des donnees

In [ ]:

centroide_csv = '/content/gdrive/MyDrive/Master II/geodata/centroide.csv'
df = pd.read_csv(centroide_csv)

In [ ]:
df = df.set_index('id')
df.head()

,lon,lat,geocode
id,,,
1,-0.993621,12.583640,550301
2,-0.286198,12.380933,480301
3,-0.847316,14.166488,590201
4,-2.019413,12.840475,540201
5,-2.635513,12.890953,540202


In [ ]:
imagery_count = df.count()[1] + 1
df.count()

/tmp/ipython-input-4181267993.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  imagery_count = df.count()[1] + 1


,0
lon,351
lat,351
geocode,351


In [ ]:

adm0_shp = gpd.read_file('/content/gdrive/MyDrive/Master II/découpage administratif/admin/bfa_admin0.shp')
adm0_shp.head()

,iso2,iso3,adm0_name,adm0_name1,adm0_name2,adm0_name3,adm0_pcode,valid_on,valid_to,version,area_sqkm,lang,lang1,lang2,lang3,adm0_ref_n,center_lat,center_lon,geometry
0,BF,BFA,Burkina Faso,Burkina Faso,None,None,BF,2025-08-01,NaT,v03,273209.239602,en,fr,None,None,None,NaN,NaN,"POLYGON ((-0.57615 15.08258, -0.56666 15.08252..."


In [ ]:
format="EPSG:32630"
geo = adm0_shp.to_crs(format)

bbox_poly = geo.geometry.envelope
bbox = bbox_poly.to_json()

In [ ]:
print(geo)
print("eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee")
print(bbox_poly,)
print("rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr")
print(bbox)

  iso2 iso3     adm0_name    adm0_name1 adm0_name2 adm0_name3 adm0_pcode  \
0   BF  BFA  Burkina Faso  Burkina Faso       None       None         BF   

    valid_on valid_to version      area_sqkm lang lang1 lang2 lang3  \
0 2025-08-01      NaT     v03  273209.239602   en    fr  None  None   

  adm0_ref_n  center_lat  center_lon  \
0       None         NaN         NaN   

                                            geometry  
0  POLYGON ((760549.566 1668894.651, 761570.077 1...  
eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
0    POLYGON ((225065.088 1040779.586, 1089672.879 ...
dtype: geometry
rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr
{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {}, "geometry": {"type": "Polygon", "coordinates": [[[225065.08815499116, 1040779.58555298], [1089672.8791363644, 1040779.58555298], [1089672.8791363644, 1669078.5536281124], [225065.08815499116, 1669078.5536281124], [225065.08815499116,

In [ ]:
# Extraction des coordonnees dans le json
#bbox_dict = json.loads(bbox)
bbox_dict = eval(bbox)
bbox_features_dict = bbox_dict['features'][0]

bbox_coordinates = bbox_features_dict['geometry']['coordinates']


In [ ]:
# Convertissez les coordonnées de la boîte englobante en un objet polygonal GEE.
bounding_box = ee.Geometry.Polygon(bbox_coordinates)

In [ ]:
print(bbox_dict)
print("------------------------------------------------------------------------------")
print(bbox_features_dict)
print("------------------------------------------------------------------------------")
print(bbox_coordinates)
print("------------------------------------------------------------------------------")
print(bounding_box)
print("------------------------------------------------------------------------------")

{'type': 'FeatureCollection', 'features': [{'id': '0', 'type': 'Feature', 'properties': {}, 'geometry': {'type': 'Polygon', 'coordinates': [[[225065.08815499116, 1040779.58555298], [1089672.8791363644, 1040779.58555298], [1089672.8791363644, 1669078.5536281124], [225065.08815499116, 1669078.5536281124], [225065.08815499116, 1040779.58555298]]]}, 'bbox': [225065.08815499116, 1040779.58555298, 1089672.8791363644, 1669078.5536281124]}], 'bbox': [225065.08815499116, 1040779.58555298, 1089672.8791363644, 1669078.5536281124], 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::32630'}}}
------------------------------------------------------------------------------
{'id': '0', 'type': 'Feature', 'properties': {}, 'geometry': {'type': 'Polygon', 'coordinates': [[[225065.08815499116, 1040779.58555298], [1089672.8791363644, 1040779.58555298], [1089672.8791363644, 1669078.5536281124], [225065.08815499116, 1669078.5536281124], [225065.08815499116, 1040779.58555298]]]}, 'bbox': [2

# **Filter et Visualiser les images satellitaire**

Consultez l'image composite pour vérifier si le filtre temporel utilisé permettra de générer une image complète, notamment
pour l'imagerie satellite Sentinel-2, couvrant l'ensemble du pays.

In [ ]:
country = "BFA"
year = "2010"
sat_sent = "ST"
sat_landsat="LS"
sat_sentinel_source='COPERNICUS/S2'
sat_landsat_7_source='LANDSAT/LE07/C01/T1'
sat_landsat_7="LS7"
sat_landsat_8_source='LANDSAT/LC08/C01/T1'
sat_landsat_8="LS8"

In [ ]:

if int(year) >= 2015:
  day_sat = sat_sent
  img_res = "384"
  img_size = str(int(img_res)*10)
else:
  day_sat = sat_landsat
  img_res = "256"
  img_size = str(int(img_res)*15)

# Ton répertoire de base
base_path = '/content/gdrive/MyDrive/Master II/data'

# Le nom du dossier spécifique (ton code original)
folder_name = "_".join(["CNN", "IMGB", country, year, day_sat, img_res, "TIF", img_size])

# Chemin complet vers le dossier de sortie
drive_folder = os.path.join(base_path, folder_name)

#generer le repertoire de sortie
#drive_folder = "_".join(["CNN", "IMGB", country,year, day_sat, img_res,"TIF", img_size])

#assemble DIMG filname
DIMG = "_".join(["CNN_IMGB", country,year, day_sat, img_res, img_size])

print(day_sat)
print(img_res)
print(img_size)
print(drive_folder)
print(DIMG)

LS
256
3840
/content/gdrive/MyDrive/Master II/data/CNN_IMGB_BFA_2010_LS_256_TIF_3840
CNN_IMGB_BFA_2010_LS_256_3840


### Précisez la date de début de la couverture par imagerie satellite.texte en gras

In [ ]:
start_MM = "01"
start_DD = "01"
start_date = "-".join([year, start_MM, start_DD])
print("Debut de la couverture "+start_date)

end_YYYY = year
end_MM = "12"
end_DD = "31"
if int(end_YYYY)<int(year):
  end_YYYY=str(int(year)+1)
  print("Please specify", end_YYYY)

end_date = "-".join([end_YYYY, end_MM, end_DD])
print("Fin de la couverture "+end_date)

Debut de la couverture 2010-01-01
Fin de la couverture 2010-12-31


Visualisez les images pour vérifier si le filtre temporel permet d'obtenir des images complètes pour l'ensemble du pays.
Commencez par importer la bibliothèque Folium dans votre environnement Python. Folium est une bibliothèque Python de visualisation de données géospatiales.

Pour les images satellites Landsat, utilisons Landsat 7 pour les images disponibles antérieures à 2013 et Landsat 8 pour les images disponibles à partir de 2013

Fusion des images Landsat (Pansharpening). Cette étape intermédiaire de préparation des données vise à améliorer la résolution des images.

In [ ]:
if day_sat == sat_sent:
  def maskS2clouds(image):
    # Récupérer la liste des noms de bandes présentes dans l'image
    band_names = image.bandNames()

    # Stratégie 1 : Si QA60 est présent (Ancienne méthode)
    def mask_qa60():
        qa = image.select('QA60')
        cloudBitMask = 1 << 10
        cirrusBitMask = 1 << 11
        return qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    # Stratégie 2 : Si MSK_CLASSI est présent (Nouvelle méthode)
    def mask_msk():
        return image.select('MSK_CLASSI_OPAQUE').eq(0).And(image.select('MSK_CLASSI_CIRRUS').eq(0))

    # On utilise ee.Algorithms.If pour choisir la bonne méthode dynamiquement dans le cloud
    is_qa60 = band_names.contains('QA60')

    # Création du masque selon la disponibilité des bandes
    mask = ee.Image(ee.Algorithms.If(is_qa60, mask_qa60(), mask_msk()))

    # Retourner l'image masquée et mise à l'échelle
    return image.updateMask(mask).divide(10000)


  rgbVis = {'min':0.0, 'max':0.3, 'bands':['B4','B3','B2']}

  #Filtre une collection d'image
  cloud_mask = ee.ImageCollection(sat_sentinel_source).filterDate(start_date,end_date).filterBounds(bounding_box).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60)).map(maskS2clouds)
  # Valeur median
  satellite_imagery = cloud_mask.median().visualize(**rgbVis)
else :
  if int(year) < 2013:
    landsat_mission = sat_landsat_7_source
    LS_day_sat = sat_landsat_7
  else:
    landsat_mission = sat_landsat_8_source
    LS_day_sat = sat_landsat_8
  #Appliquer le filtre à la collection d'images Landsat sélectionnée.
  filtered_shp = ee.ImageCollection(landsat_mission).filterDate(start_date, end_date).filterBounds(bounding_box)

  #Générez une image composite pour l'ensemble du pays à partir de la collection d'images filtrée
  composite = ee.Algorithms.Landsat.simpleComposite(filtered_shp).float();

  #Pansharpening : fusion panchromatique
  if LS_day_sat == sat_landsat_7:
    rgb = composite.select('B3', 'B2', 'B1').unitScale(0, 255)

  elif LS_day_sat == sat_landsat_8:
    rgb = composite.select('B4', 'B3', 'B2').unitScale(0, 255)

  gray = composite.select('B8').unitScale(0, 155)

  #Convertissez l'image RGB en teinte, saturation et valeur (HSV) et
  #sélectionnez uniquement les bandes de teinte et de saturation.
  huesat = rgb.rgbToHsv().select('hue', 'saturation')

  #Combinez la teinte, la saturation et les bandes panchromatiques.
  #Convertissez ensuite le résultat en RGB pour obtenir l'image agrandie.
  satellite_imagery = ee.Image.cat(huesat, gray).hsvToRgb().multiply(255).byte()


# Déterminez les coordonnées x et y du centroïde du polygone englobant la boîte.

In [ ]:
#bbox_centroid = bounding_box.centroid()
#cen_x = bbox_centroid.coordinates().get(0).getInfo()
#cen_y = bbox_centroid.coordinates().get(1).getInfo()

In [ ]:
#Déterminez les coordonnées x et y du centroïde du polygone englobant la boîte
cen_x = bbox_poly.centroid.x[0]
cen_y = bbox_poly.centroid.y[0]

#Créez un objet carte Folium
map = folium.Map(location=[cen_x, cen_y],
                 zoom_start=6,
                 width=1280,
                 height=766,
                 attr=day_sat)

In [ ]:
#from os import name
#from numpy._core.defchararray import title

# Définir les paramètres de visualisation (min, max, gamma)
#vis_params = {'min': 0, 'max': 0.3, 'gamma': 1.4}

#l' identifiant de la carte (mapID) de l'imagerie satellite filtrée.
ee_image_map_id = ee.Image(satellite_imagery).getMapId()
print(ee_image_map_id)


#Générez une nouvelle couche cartographique pour visualiser les paramètres
tile_url = ee_image_map_id['tile_fetcher'].url_format
folium.raster_layers.TileLayer(
    tiles= tile_url,
    attr='Google Earth Engine',
    name='Daytime imagery',
    overlay=True,
    control=True
).add_to(map)

EEException: ImageCollection.load: ImageCollection asset 'LANDSAT/LE07/C01/T1' not found (does not exist or caller does not have access).

In [ ]:
#Superposez le polygone de la boîte englobante
folium.GeoJson(
    data=bounding_box.getInfo(),
    name='Bounding box',
    style_function=lambda feature : {
        'fillColor':'#FFFFFF00',
        'weight':3,
        'fillOpacity':0.5,
        },
    overlay=True,
    control=True,
).add_to(map)

Définissez le titre de la carte pour les images Sentinel et Landsat. Insérez un rappel pour vérifier si l'imagerie satellite générée est complète.

In [ ]:
#ajouter un titre
if day_sat == sat_sent:
  map_title = "Sentinel-2 Imagery "
else:
  map_title = "Landsat Imagery"

title_html = '''
            <h3 align="center" style="font-size:16px"><b>{}</b></h3>
            '''.format(map_title)
map.get_root().html.add_child(folium.Element(title_html))

In [ ]:
map.add_child(folium.LayerControl())
display(map)

## Utiliser une fonction qui identifie le nombre de tache

Implémentez la fonction `get_queued_task_filenames()` pour obtenir les noms de fichiers des tâches « Prêtes » et « En cours
d’exécution » de la liste des tâches GEE. Cette fonction est nécessaire pour éviter la duplication des fichiers.

In [ ]:
def get_queued_task():
  queued_task_count = 0
  for queue_task in ee.batch.Task.list():
    if queue_task.status() in['RUNNING', 'READY']:
      queued_task_count += 1
  return queued_task_count

def get_queued_task_filenames():
  print("Fetching queued files")
  task_filename = []
  for queued_task in ee.batch.Task.list():
    if queued_task.status in ['RUNNING', 'READY']:
      print(queued_task.state+": "+queued_task.status()['Description'])
      task_filename.append(queued_task.status()['Description'])
  print("-----end fetch-------")
  return task_filename

In [ ]:
def get_queued_task():
    tasks = ee.batch.Task.list()
    # On compte les tâches en attente ou en cours
    active = [t for t in tasks if t.state in ['READY', 'RUNNING']]
    return len(active)

def get_queued_task_filenames():
    print("Récupération de la file d'attente...")
    tasks = ee.batch.Task.list()
    # On extrait la description (nom du fichier)
    task_filenames = [t.config['description'] for t in tasks if t.state in ['READY', 'RUNNING']]
    print(f"----- {len(task_filenames)} fichiers en attente -----")
    return task_filenames

## Définir la fonction de téléchargement des images satellite.**texte en gras**

In [ ]:

import os
import time
def download_satellite_imagery(sat_imagery):
  if not isinstance(sat_imagery, ee.Image):
        print("Erreur: L'objet fourni n'est pas une image Earth Engine.")
        return
  next_batch_size = 10
  target_count = 3000 - next_batch_size
  task_count = get_queued_task()
  queued_filenames = get_queued_task_filenames()
  print(f'Nombre de tâches actives: {target_count}')

  for i in range(1, imagery_count):
    imagery_file = f"{DIMG}_{i:06d}"
    imagery_filepath = os.path.join('/content/gdrive/MyDrive/Master II/data', drive_folder, f"{imagery_file}.tif")
    #imagery_file = DIMG + '_{:06d}'.format(i)
    #imagery_filepath = '/content/gdrive/MyDrive/Master II/data' + drive_folder + '/'+ imagery_file + '.tif'
    if task_count == 3000:
      while task_count > target_count:
        time.sleep(30)
        active_task = get_queued_task()
        if active_task < task_count:
          task_count = active_task
          print('-------------------------------------------------')
          print('Number of current pending taskin queue: {: }.'.format(task_count))
          print('Remainig task before starting new bath : {: }.'.format(task_count-target_count))
    else:
      if not os.path.exists(imagery_filepath):
        if(imagery_file not in queued_filenames):
           print('-------------------------------------------------')
           print('Start new task')
           print('downloading : {: }.'+ imagery_file)
           c_lon = df['lon'][i]
           c_lat = df['lat'][i]
           geometry = ee.Geometry.Point([c_lon, c_lat]).buffer(1992).bounds()
           geometry = geometry.getInfo()['coordinates'][0]

           scale = 10 if day_sat == sat_sent else 15
           task = ee.batch.Export.image.toDrive(
                    image=sat_imagery,
                    description=imagery_file,
                    folder=drive_folder,
                    scale=scale,
                    region=geometry,
                    fileFormat='GeoTIFF'
                )
           #task_config = {
           #    'scale': scale,
             #  'region': geometry,
             #  'driveFolder': drive_folder,
          # }
           # Dans ta fonction d'export, modifie sat_imagery comme ceci :
           #export_ready = sat_imagery.visualize(min=0, max=0.3).copyProperties(sat_imagery)

            # Puis utilise export_ready dans ton export :
           #task = ee.batch.Export.image(export_ready, imagery_file, task_config)
           #task = ee.batch.Export.image(sat_imagery, imagery_file, task_config)
           task.start()
           task_count +=1
           if(task_count % 1000)==0:
            task_count = get_queued_task()
            print('-------------------------------------------------')
            print('Number of active task : {: }.'.format(task_count))
        else:
          print('On queue:'+ imagery_file + 'tif')
      else:
        print('Downloaded:'+ imagery_file + 'tif')


In [ ]:
# Lancement de l'exportation massive
download_satellite_imagery(satellite_imagery)

Récupération de la file d'attente...
----- 0 fichiers en attente -----
Nombre de tâches actives: 2990
-------------------------------------------------
Start new task
downloading : {: }.CNN_IMGB_BFA_2020_ST_384_3840_000001
-------------------------------------------------
Start new task
downloading : {: }.CNN_IMGB_BFA_2020_ST_384_3840_000002
-------------------------------------------------
Start new task
downloading : {: }.CNN_IMGB_BFA_2020_ST_384_3840_000003
-------------------------------------------------
Start new task
downloading : {: }.CNN_IMGB_BFA_2020_ST_384_3840_000004
-------------------------------------------------
Start new task
downloading : {: }.CNN_IMGB_BFA_2020_ST_384_3840_000005
-------------------------------------------------
Start new task
downloading : {: }.CNN_IMGB_BFA_2020_ST_384_3840_000006
-------------------------------------------------
Start new task
downloading : {: }.CNN_IMGB_BFA_2020_ST_384_3840_000007
-------------------------------------------------
S

In [ ]:
import os
import time

def download_satellite_imagery(sat_imagery):
    # 1. Vérification de sécurité
    if not isinstance(sat_imagery, ee.Image):
        print("❌ Erreur: L'objet fourni n'est pas une image Earth Engine.")
        return

    # 2. Initialisation des compteurs
    task_count = get_queued_task()
    queued_filenames = get_queued_task_filenames()

    # On définit une limite de sécurité pour GEE (généralement 3000 tâches max)
    MAX_TASKS = 3000
    TARGET_FREE_SLOTS = 100

    print(f'🚀 Démarrage de l\'exportation. Tâches actives : {task_count}')

    # 3. Boucle sur votre DataFrame
    for i in range(1, imagery_count):
        imagery_file = f"{DIMG}_{i:06d}"
        imagery_filepath = os.path.join('/content/gdrive/MyDrive/Master II/data', drive_folder, f"{imagery_file}.tif")

        # Gestion de la file d'attente (Wait if queue is full)
        if task_count >= MAX_TASKS:
            print(f"⏳ File d'attente pleine ({task_count}). En attente de libération...")
            while task_count > (MAX_TASKS - TARGET_FREE_SLOTS):
                time.sleep(60) # Attendre 1 minute
                task_count = get_queued_task()
            print(f"✅ Reprise du téléchargement. Tâches actives : {task_count}")

        # Vérification si fichier déjà présent ou en cours
        if not os.path.exists(imagery_filepath):
            if imagery_file not in queued_filenames:
                try:
                    # Récupération coordonnées
                    c_lon = float(df['lon'][i])
                    c_lat = float(df['lat'][i])

                    # Création de la géométrie (SANS getInfo pour plus de stabilité)
                    geometry = ee.Geometry.Point([c_lon, c_lat]).buffer(1992).bounds()

                    # Définition de l'échelle
                    scale_val = 10 if day_sat == sat_sent else 15

                    # Configuration de l'export
                    task = ee.batch.Export.image.toDrive(
                        image=sat_imagery,
                        description=imagery_file,
                        folder=drive_folder,
                        scale=scale_val,
                        region=geometry,
                        fileFormat='GeoTIFF',
                        maxPixels=1e13 # Pour éviter les erreurs de limites de pixels
                    )

                    task.start()
                    task_count += 1

                    if i % 10 == 0:
                        print(f"Envoi en cours : {imagery_file} ({i}/{imagery_count})")

                except Exception as e:
                    print(f"⚠️ Erreur sur l'image {i}: {e}")
            else:
                pass # Déjà dans la file d'attente GEE
        else:
            pass # Déjà sur le Drive

In [ ]:
# Juste avant l'appel :
print(f"DEBUG: Type de l'image = {type(satellite_imagery)}")
# Si cela n'affiche pas <class 'ee.image.Image'>,
# c'est que vous avez écrasé la variable plus haut dans votre code.

download_satellite_imagery(satellite_imagery)

DEBUG: Type de l'image = <class 'ee.image.Image'>
Récupération de la file d'attente...
----- 0 fichiers en attente -----
🚀 Démarrage de l'exportation. Tâches actives : 0
Envoi en cours : CNN_IMGB_BFA_2020_ST_384_3840_000010 (10/352)
Envoi en cours : CNN_IMGB_BFA_2020_ST_384_3840_000020 (20/352)
Envoi en cours : CNN_IMGB_BFA_2020_ST_384_3840_000030 (30/352)
Envoi en cours : CNN_IMGB_BFA_2020_ST_384_3840_000040 (40/352)
Envoi en cours : CNN_IMGB_BFA_2020_ST_384_3840_000050 (50/352)
Envoi en cours : CNN_IMGB_BFA_2020_ST_384_3840_000060 (60/352)
Envoi en cours : CNN_IMGB_BFA_2020_ST_384_3840_000070 (70/352)
Envoi en cours : CNN_IMGB_BFA_2020_ST_384_3840_000080 (80/352)
Envoi en cours : CNN_IMGB_BFA_2020_ST_384_3840_000090 (90/352)
Envoi en cours : CNN_IMGB_BFA_2020_ST_384_3840_000100 (100/352)
Envoi en cours : CNN_IMGB_BFA_2020_ST_384_3840_000110 (110/352)
Envoi en cours : CNN_IMGB_BFA_2020_ST_384_3840_000120 (120/352)
Envoi en cours : CNN_IMGB_BFA_2020_ST_384_3840_000130 (130/352)
Envoi e

In [ ]:
# Récupérer la liste de toutes les tâches
tasks = ee.batch.Task.list()

count = 0
for task in tasks:
    # Vérifier si la tâche est en cours ou en attente
    if task.state in ['READY', 'RUNNING']:
        task.cancel()
        count += 1

print(f"🛑 {count} tâches ont été annulées avec succès.")

🛑 342 tâches ont été annulées avec succès.
